In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [3]:
from langchain_community.document_loaders import UnstructuredPDFLoader

In [4]:
file_path = "../data/raw/HSC26-Bangla1st-Paper.pdf"

In [5]:
loader = UnstructuredPDFLoader(
    file_path,
    strategy="ocr_only",
    languages=["ben"],
)

In [6]:
docs = loader.load()

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [10]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=10000, chunk_overlap=100)
splits = text_splitter.split_documents(docs)

In [12]:
import uuid

from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="moonshotai/kimi-k2-instruct",
)
chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("Summarize the following document:\n\n{doc}")
    | llm
    | StrOutputParser()
)

summaries = chain.batch(splits, {"max_concurrency": 10})

In [13]:
summaries

['**সারাংশ: “অপরিচিতা”**\n\n“অপরিচিতা” একটি উপন্যাসের অনুচ্ছেদ, যেখানে কথক অনুপম তার নিজের জীবন, পারিবারিক পটভূমি এবং একটি সম্ভাব্য বিবাহের প্রস্তাব নিয়ে আত্মসমালোচনা করছে।  \n\n**মূল বিষয়বস্তু:**\n\n- **অনুপমের পরিচয়**: সে ২৭ বছর বয়সী, স্নাতক, তবে জীবনে গুণ বা দৈর্ঘ্যের দিক দিয়ে তেমন কিছু অর্জন করেনি।  \n- **পারিবারিক পটভূমি**: তার পিতা একসময় গরিব ছিলেন, পরে ওকালতি করে ধনী হয়ে যান। মা গরিব ঘরের মেয়ে, এবং মামাই তার ভাগ্যদেবতার প্রধান এজেন্ট।  \n- **বিবাহের প্রস্তাব**: অনুপমের বন্ধু হরিশ তার কলিকাতা আসার পর এক বিবাহ প্রস্তাবের আলোচনা শুরু হয়। মেয়েটির বয়স ১৫, তার পিতা একসময় ধনী ছিলেন, এখন অর্থাভাব। মেয়ের মাথা হেট করে আসবে, তবে যৌতুক দেওয়ার সামর্থ্য নেই।  \n- **মামার মতামত**: তিনি চান না অনুপম ধনীর মেয়েকে বিয়ে করুক। মেয়েটির পিতার অর্থ না থাকলেও মানসম্মান আছে, তাই বিবাহের সিদ্ধান্ত হয়।  \n- **বিনুদাদা**: অনুপমের পিসতুতো ভাই, তাকে পাঠানো হয় মেয়েটিকে দেখে আসতে।  \n\n**শিক্ষণফল ও প্রশ্নোত্তরের সারাংশ**:  \n- শিক্ষার্থীরা গল্প থেকে সমাজে পরিচয় সংকট, পণপ্রথা, নারীর সাহসিকতা

In [15]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large-instruct")

In [16]:
from langchain.storage import InMemoryByteStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.retrievers.multi_vector import MultiVectorRetriever

vectorstore = Chroma(collection_name="summaries",
                     embedding_function=embeddings)

store = InMemoryByteStore()
id_key = "doc_id"

# The retriever
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)
doc_ids = [str(uuid.uuid4()) for _ in splits]

summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]

# Add
retriever.vectorstore.add_documents(summary_docs)
retriever.docstore.mset(list(zip(doc_ids, splits)))

C:\Users\mdsaz\AppData\Local\Temp\ipykernel_17428\3364906949.py:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(collection_name="summaries",


In [17]:
query = "অপরিচিতা' গল্পে, অনুপমের মতে কে আসর জমাতে অদ্বিতীয়?"
sub_docs = vectorstore.similarity_search(query, k=1)
sub_docs[0]

Document(metadata={'doc_id': 'cb9fd1ce-4cd2-4b28-baa4-cd59bb31c52d'}, page_content='**সারাংশ: “অপরিচিতা”**\n\n“অপরিচিতা” একটি উপন্যাসের অনুচ্ছেদ, যেখানে কথক অনুপম তার নিজের জীবন, পারিবারিক পটভূমি এবং একটি সম্ভাব্য বিবাহের প্রস্তাব নিয়ে আত্মসমালোচনা করছে।  \n\n**মূল বিষয়বস্তু:**\n\n- **অনুপমের পরিচয়**: সে ২৭ বছর বয়সী, স্নাতক, তবে জীবনে গুণ বা দৈর্ঘ্যের দিক দিয়ে তেমন কিছু অর্জন করেনি।  \n- **পারিবারিক পটভূমি**: তার পিতা একসময় গরিব ছিলেন, পরে ওকালতি করে ধনী হয়ে যান। মা গরিব ঘরের মেয়ে, এবং মামাই তার ভাগ্যদেবতার প্রধান এজেন্ট।  \n- **বিবাহের প্রস্তাব**: অনুপমের বন্ধু হরিশ তার কলিকাতা আসার পর এক বিবাহ প্রস্তাবের আলোচনা শুরু হয়। মেয়েটির বয়স ১৫, তার পিতা একসময় ধনী ছিলেন, এখন অর্থাভাব। মেয়ের মাথা হেট করে আসবে, তবে যৌতুক দেওয়ার সামর্থ্য নেই।  \n- **মামার মতামত**: তিনি চান না অনুপম ধনীর মেয়েকে বিয়ে করুক। মেয়েটির পিতার অর্থ না থাকলেও মানসম্মান আছে, তাই বিবাহের সিদ্ধান্ত হয়।  \n- **বিনুদাদা**: অনুপমের পিসতুতো ভাই, তাকে পাঠানো হয় মেয়েটিকে দেখে আসতে।  \n\n**শিক্ষণফল ও প্রশ্নোত্তরে

In [18]:
retrieved_docs = retriever.get_relevant_documents(query, n_results=1)
retrieved_docs[0].page_content[0:500]

C:\Users\mdsaz\AppData\Local\Temp\ipykernel_17428\494825069.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query, n_results=1)


'অপরিচিতা\n\nঅনলাইন ব্যাচ সম্পর্কিত যেকোনো জিজ্ঞাসায়,\n\nকলকরো ৬ 16919\n\n[ছল নী নেও\n\nঅনলাইন ব্যাচ 101৫159: ভ্ শিখনফল\n\n৮ নিম্নবিত্ত ব্যক্তির হঠাৎ বিত্তশালী হয়ে ওঠার ফলে সমাজে পরিচয় সংকট সম্পর্কে ধারণা লাভ করবে।\n\n৮ তৎকালীন সমাজ-সভ্যতা ও মানবতার অবমাননা সম্পর্কে জানতে পারবে।\n\n৮ তৎকালীন সমাজের পণপ্রথার কুপ্রভাব সম্পর্কে জানতে পারবে।\n\n৮ তৎকালে সমাজে ভদ্রলোকের স্বভাববৈশিষ্ট্য সম্পর্কে জ্ঞানলাভ করবে। ৮ নারী কোমল ঠিক, কিন্তু দুর্বল নয়- কল্যাণীর জীবনচরিত দ্বারা প্রতিষ্ঠিত এই সত্য অনুধাবন করতে\n\nপারবে। ৮ মান'